In [2]:
import os
if os.getcwd().split("/")[-1] != 'BENDR-XAI': os.chdir("../")

import mne
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation
import random
import matplotlib.cm as cm
import sys
from tqdm import tqdm

In [7]:
# Make a list of path to all edf files in the subfolders of /nobackup/tsal-tmp/tuh_eeg
# files = []

# with tqdm() as pbar:
#     for root, dirs, file in os.walk("/nobackup/tsal-tmp/tuh_eeg"):
#         for name in file:
#             if name.endswith(".edf"):
#                 files.append(os.path.join(root, name))
#                 pbar.update(1)

# Save list of files as pickle
# import pickle
# with open('tuh_files.pkl', 'wb') as f:
#     pickle.dump(files, f)
    
# Load list of files from pickle
# import pickle
# with open('tuh_files.pkl', 'rb') as f:
#     files = pickle.load(f)
    
# random.shuffle(files)

In [8]:
selected = []
with open('tuh_final_selected.txt', 'r') as f:
    selected.extend(f.read().splitlines())

deselected = []
with open('tuh_final_deselected.txt', 'r') as f:
    deselected.extend(f.read().splitlines())

# Open all the files tuh_selected*.txt, read the filesnames on each line and save them in one list
files = []
# for file in os.listdir():
#     if file.startswith("tuh_selected"):
#         with open(file, 'r') as f:
#             files_selected.extend(f.read().splitlines())
files = ['data/tuh_eeg/' + file for file in os.listdir('data/tuh_eeg/')]
files = [file for file in files if file not in selected and file not in deselected]

In [9]:
# Delete all files in data/tuh_eeg/ that are not in selected
for file in os.listdir('data/tuh_eeg/'):
    if file not in selected and file not in deselected:
        os.remove('data/tuh_eeg/' + file)

In [5]:
%matplotlib auto
import mne
import matplotlib.pyplot as plt

# Read all files in tuh_final_selected.txt

# Function to handle user input
def on_key(event):
    if event.key == 'y':
        with open('tuh_final_selected.txt', 'a') as f:
            f.write(current_file + '\n')
        
        plt.close()
    elif event.key == 'n':
        with open('tuh_final_deselected.txt', 'a') as f:
            f.write(current_file + '\n')
        
        plt.close()

# Iterate through the EDF files
for file_path in files:
    
    if file_path in selected or file_path in deselected:
        continue   
    
    # Load the EDF file using MNE
    raw = mne.io.read_raw_edf(file_path, preload=True, verbose=False)
    raw.resample(256, verbose=False)
    raw.filter(1, 70, fir_design='firwin', verbose=False)
    raw.notch_filter(60, fir_design='firwin', verbose=False)
    
    # Plot the data
    fig = raw.plot(show=False, verbose=False, scalings='auto')
    current_file = file_path
    plt.title(f'File: {file_path}\nPress "y" to save or any other key to skip')
    
    # Register the event handler and show the plot
    cid = fig.canvas.mpl_connect('key_press_event', on_key)
    plt.show()

Using matplotlib backend: <object object at 0x7f84ac0d8360>
Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none


In [5]:
# import pickle
# with open('tuh_files.pkl', 'rb') as f:
#     files = pickle.load(f)

# Copy all files in tuh_final_selected.txt to data/tuh_eeg/
# with open('tuh_final_selected.txt', 'r') as f:
#     selected = f.read().splitlines()

# selected = [file.split("/")[-1] for file in selected]

# Delete all files in /scratch/s194260/tuh_eeg/ that are not in selected
# for file in os.listdir('/scratch/s194260/tuh_eeg/'):
#     if file not in selected:
#         os.remove('/scratch/s194260/tuh_eeg/' + file)

In [5]:
# Get length of all edf files in /scratch/s194260/tuh_eeg/
lengths = []
for file in tqdm(os.listdir('/scratch/s194260/tuh_eeg/')):
    raw = mne.io.read_raw_edf('/scratch/s194260/tuh_eeg/' + file, preload=True, verbose=False)
    raw.resample(256, verbose=False)
    lengths.append(raw.n_times)

  0%|          | 0/200 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [3]:
import mne
import numpy as np
import matplotlib.pyplot as plt
from notebooks.utils import *

from matplotlib import animation
import matplotlib.cm as cm
import sys
from tqdm import tqdm
from pathlib import Path
import multiprocessing
import argparse

In [9]:
list(channel_positions.values())

['Fp1',
 'Fp2',
 'F3',
 'F4',
 'C3',
 'C4',
 'P3',
 'P4',
 'O1',
 'O2',
 'F7',
 'F8',
 'T7',
 'T8',
 'P7',
 'P8',
 'FT9',
 'FT10',
 'Fz',
 'Pz',
 'Cz']

In [8]:
file_path = '/scratch/s194260/tuh_eeg/' + file

raw = mne.io.read_raw_edf(file_path, preload=True, verbose=True)
mne.datasets.eegbci.standardize(raw)
raw = raw.set_eeg_reference(ref_channels='average', projection=True, verbose = False)

# Channel positions dictionary (subset)
channel_positions = {
    'EEG FP1-REF': 'Fp1',
    'EEG FP2-REF': 'Fp2',
    'EEG F3-REF': 'F3',
    'EEG F4-REF': 'F4',
    'EEG C3-REF': 'C3',
    'EEG C4-REF': 'C4',
    'EEG P3-REF': 'P3',
    'EEG P4-REF': 'P4',
    'EEG O1-REF': 'O1',
    'EEG O2-REF': 'O2',
    'EEG F7-REF': 'F7',
    'EEG F8-REF': 'F8',
    'EEG T3-REF': 'T7',
    'EEG T4-REF': 'T8',
    'EEG T5-REF': 'P7',
    'EEG T6-REF': 'P8',
    'EEG T1-REF': 'FT9',
    'EEG T2-REF': 'FT10',
    'EEG FZ-REF': 'Fz',
    'EEG CZ-REF': 'Cz',
    'EEG PZ-REF': 'Pz',
}   


# Create the custom montage
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')

# Rename channels and set the custom montage
raw = raw.rename_channels(channel_positions)
raw = raw.set_montage(ten_twenty_montage, on_missing='ignore')

# Set channel types for non-EEG channels
non_eeg_channels = {
    'EEG EKG1-REF': 'ecg',
    'EMG-REF': 'emg',
    'PHOTIC-REF': 'misc',
    'IBI': 'misc',
    'BURSTS': 'misc',
    'SUPPR': 'misc',
    'EEG A1-REF': 'eog',
    'EEG A2-REF': 'eog',
    'EEG 31-REF': 'eog',
    'EEG 32-REF': 'eog',
    'EEG C3P-REF': 'eog',
    'EEG C4P-REF': 'eog',
    'EEG SP1-REF': 'eog',
    'EEG SP2-REF': 'eog',
}

#raw = raw.set_channel_types(non_eeg_channels, on_missing='ignore')

Extracting EDF parameters from /scratch/s194260/tuh_eeg/aaaaamhf_s007_t026.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 77055  =      0.000 ...   300.996 secs...


In [10]:
subjects_dir, subject, trans, src_path, bem_path = get_fsaverage()
src = get_src(src_path)
fwd = get_fwd(raw.info, trans, src_path, bem_path)
cov = get_cov(raw)
compute_inverse = make_fast_inverse_operator(raw.info, fwd, cov, snr=100)

In [11]:
print(f'Length of raw: {raw.n_times / raw.info["sfreq"]} seconds')

Length of raw: 301.0 seconds


In [12]:
compute_inverse(raw)

<SourceEstimate | 20484 vertices, subject : fsaverage, tmin : 0.0 (ms), tmax : 300996.09375 (ms), tstep : 3.90625 (ms), data shape : (20484, 77056), ~11.76 GB>

In [62]:
raw = mne.io.read_raw_edf('data/eegmmidb/files/S003/S003R01.edf', verbose=False, preload=True)
# Apply average reference if not already applied
if not raw.info['custom_ref_applied']:
    raw.set_eeg_reference(ref_channels='average', projection=True, verbose = False)
    # applt projection
    raw.apply_proj()

Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


In [64]:
raw1 = mne.io.read_raw_edf('data/eegmmidb/files/S003/S003R01.edf', verbose=False, preload=True)

In [65]:
raw.get_data()

array([[5.7406250e-05, 1.1237500e-04, 1.0971875e-04, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [5.1406250e-05, 8.9375000e-05, 7.7718750e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [7.7406250e-05, 1.0637500e-04, 1.0771875e-04, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [6.2406250e-05, 6.2375000e-05, 4.7718750e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [6.5406250e-05, 6.8375000e-05, 5.5718750e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [7.3406250e-05, 6.9375000e-05, 5.3718750e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]])

In [66]:
raw1.get_data()

array([[1.53e-04, 1.77e-04, 1.30e-04, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [1.47e-04, 1.54e-04, 9.80e-05, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [1.73e-04, 1.71e-04, 1.28e-04, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       ...,
       [1.58e-04, 1.27e-04, 6.80e-05, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [1.61e-04, 1.33e-04, 7.60e-05, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [1.69e-04, 1.34e-04, 7.40e-05, ..., 0.00e+00, 0.00e+00, 0.00e+00]])

In [50]:
print(raw.info)

<Info | 8 non-empty values
 bads: []
 ch_names: EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-REF, EEG C3-REF, ...
 chs: 35 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2012-01-26 07:50:43 UTC
 nchan: 35
 projs: Average EEG reference: off
 sfreq: 256.0 Hz
>


In [52]:
print(mne.io.read_raw_edf('data/eegmmidb/files/S003/S003R01.edf', verbose=False, preload=True).info)

<Info | 7 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 80.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
>
